In [33]:
# # Load & Test Model ----> v.1
# import joblib
# import pandas as pd
# import numpy as np
# from sklearn.metrics import accuracy_score
# from memory_profiler import memory_usage
# import time

# # Load the trained model and signature list
# model_filename = "C:/Users/Natty PC/Documents/Party/Project II/Models/Model - DCT.joblib"
# loaded_data = joblib.load(model_filename)

# # Extract the classifier (model) and signature_list
# clf = loaded_data['model']  # Decision tree classifier
# signature_list = loaded_data['signature_list']  # Signature list

# print(f"Model and signature list loaded from {model_filename}")

# # Load new data (adjust file path as necessary)
# # new_data = pd.read_csv('C:/Users/Natty PC/Documents/Party/Project II/PreData/Testing Data/data-3-1.csv')
# new_data = pd.read_csv('C:/Users/Natty PC/Documents/Party/Project II/PreData/Testing Data/data-60-1.csv')

# # Print the columns to find the true label column
# print("Columns in the dataset:")
# print(new_data.columns)

# # Preprocess new data using the same signature list
# def extract_signature_and_frequency(cell):
#     if isinstance(cell, str):
#         try:
#             signature, frequency = cell.replace('[', '').replace(']', '').split(',')
#             return signature.strip(), int(frequency.strip())
#         except ValueError:
#             return None, 0  # Return None and 0 frequency on failure
#     return None, 0

# def preprocess_new_data(new_data, signature_list):
#     fs_columns = [col for col in new_data.columns if col.startswith('Fs')]
#     X_new = np.zeros((new_data.shape[0], len(signature_list)), dtype=int)

#     for row_idx in range(new_data.shape[0]):
#         for col in fs_columns:
#             signature, frequency = extract_signature_and_frequency(new_data.at[row_idx, col])
#             if signature in signature_list:
#                 sig_idx = signature_list.index(signature)
#                 X_new[row_idx, sig_idx] += frequency

#     return X_new

# # Function to test model and calculate memory usage
# def test_model():
#     # Start measuring time
#     start_time = time.time()

#     # Preprocess new data
#     X_new = preprocess_new_data(new_data, signature_list)

#     # Make predictions with the loaded model
#     y_pred_new = clf.predict(X_new)

#     # Check for the presence of true labels
#     true_label_column = 'Label'  # Ensure this matches exactly with the dataset

#     if true_label_column not in new_data.columns:
#         print(f"Error: True label column '{true_label_column}' not found in the data.")
#         return

#     # Extract true labels for accuracy calculation
#     true_labels = new_data[true_label_column].values  # Use the updated column name

#     # Map numeric predictions to class names correctly
#     label_mapping = {
#         0: 'Attack',
#         1: 'Benign',          # Ensure this aligns with your true labels
#         2: 'C&C',
#         3: 'C&C-HeartBeat',   # Align these as per your training dataset classes
#         4: 'DDoS',
#         5: 'Okiru',
#         6: 'PortScan',
#         7: 'PortScan-Attack',
#     }

#     # Convert numeric predictions to class names
#     y_pred_labels = [label_mapping.get(label, "Unknown") for label in y_pred_new]

#     # Calculate accuracy
#     accuracy = accuracy_score(true_labels, y_pred_labels)
#     # print(f"Accuracy of the model: {accuracy:.2f}")

#     # Print a comparison of predictions and true labels for the first 20 rows
#     print("\nPredicted \tActual")
#     print("------------------------")
#     for i in range(min(20, len(y_pred_new))):  # Print only up to 20 rows
#         print(f"{y_pred_labels[i]}     \t{true_labels[i]}")

#     # Show average score
#     average_score = np.mean((true_labels == y_pred_labels).astype(float))
#     print(f"\nAverage Score: {average_score:.2f}")

#     # Stop measuring time
#     end_time = time.time()
#     testing_time = end_time - start_time

#     # Print results
#     print(f"\nTesting Time: {testing_time:.10f} seconds")
#     print(f"Test Accuracy: {accuracy * 100:.2f}%")
    
#     return accuracy  # Return accuracy for final calculation

# # Measure memory usage during model testing
# mem_usage = memory_usage(test_model)
# initial_memory_usage = mem_usage[0]  # Initial memory usage

# # Final memory usage
# final_memory_usage = mem_usage[-1]  # Final memory usage
# memory_consumption_during_testing = final_memory_usage - initial_memory_usage

# # Print memory consumption results
# print(f"\nInitial Memory Usage: {initial_memory_usage:.2f} KiB")
# print(f"Final Memory Usage: {final_memory_usage:.2f} KiB")
# print(f"Memory Consumption During Testing: {memory_consumption_during_testing:.2f} KiB")


In [ ]:
# import pandas as pd
# import joblib
# import numpy as np
# import os

# def generate_signatures_from_log(df):
#     """
#     Generate signatures from the DataFrame of logs.
    
#     Parameters:
#         df (DataFrame): DataFrame containing the log data.
        
#     Returns:
#         DataFrame: A DataFrame with signatures and their frequencies.
#     """
#     # Initialize a dictionary to hold the frequency of each signature
#     signature_counts = {}

#     # Create signatures based on specified criteria
#     for _, row in df.iterrows():
#         # Only include 'service' if it's not '-'
#         service = row['service'] if row['service'] != '-' else ''
        
#         # Create the signature string
#         signature = f"{row['id.resp_p']}_{row['proto']}_{service}_{row['orig_bytes']}_{row['resp_bytes']}_{row['conn_state']}"
        
#         # Count frequencies
#         if signature in signature_counts:
#             signature_counts[signature] += 1
#         else:
#             signature_counts[signature] = 1

#     # Prepare the output list in the desired format
#     output_list = [[signature, frequency] for signature, frequency in signature_counts.items()]

#     # Convert the output list to a DataFrame for easy processing
#     return pd.DataFrame(output_list, columns=['signature', 'frequency'])


# def process_log_in_intervals(file_path, model_path, interval_minutes=5):
#     """
#     Read the log file in 5-minute intervals and predict using the trained model.
    
#     Parameters:
#         file_path (str): Path to the log file.
#         model_path (str): Path to the trained model.
#         interval_minutes (int): Interval duration in minutes.
#     """
#     try:
#         # Load the log file into a DataFrame
#         df = pd.read_csv(file_path, delim_whitespace=True, header=None,
#                          usecols=[0, 5, 6, 7, 9, 10, 11], names=[
#                              'ts', 'id.resp_p', 'proto', 'service', 
#                              'orig_bytes', 'resp_bytes', 'conn_state'],
#                          na_filter=False)

#         # Replace '-' or '0' in orig_bytes and resp_bytes with '0'
#         df['orig_bytes'] = df['orig_bytes'].replace({'-': '0', '0': '0'})
#         df['resp_bytes'] = df['resp_bytes'].replace({'-': '0', '0': '0'})

#         # Convert 'ts' to datetime
#         df['ts'] = pd.to_datetime(df['ts'], unit='s', errors='coerce')

#         # Check for missing values and drop them
#         df.dropna(subset=['id.resp_p', 'proto', 'conn_state', 'ts'], inplace=True)

#         # Load the trained model
#         model_data = joblib.load(model_path)
#         clf = model_data['model']
#         signature_list = model_data['signature_list']

#         # Define time intervals
#         start_time = df['ts'].min()
#         end_time = df['ts'].max()
#         current_time = start_time

#         while current_time <= end_time:
#             # Create time range for the current 5-minute interval
#             next_time = current_time + pd.Timedelta(minutes=interval_minutes)

#             # Filter the DataFrame for the current time range
#             filtered_df = df[(df['ts'] >= current_time) & (df['ts'] < next_time)]

#             if not filtered_df.empty:
#                 # Generate signatures from the filtered DataFrame
#                 signatures_df = generate_signatures_from_log(filtered_df)

#                 # Prepare the signature matrix for prediction
#                 X_matrix = np.zeros((1, len(signature_list)), dtype=int)
                
#                 # Populate the matrix with frequencies
#                 for _, row in signatures_df.iterrows():
#                     signature = row['signature']
#                     frequency = row['frequency']
#                     if signature in signature_list:
#                         sig_idx = signature_list.index(signature)  # Get the index for the signature
#                         X_matrix[0, sig_idx] += frequency  # Update the corresponding cell with the frequency

#                 # Make predictions
#                 prediction = clf.predict(X_matrix)

#                 # Print results
#                 print(f"Time Interval: {current_time} - {next_time}")
#                 print(f"Signatures: {signatures_df}")
#                 print(f"Predicted Label: {prediction[0]}")

#             # Move to the next time interval
#             current_time = next_time

#     except FileNotFoundError:
#         print(f"File not found: {file_path} or {model_path}")
#     except Exception as e:
#         print(f"An error occurred: {e}")

# # Example usage
# log_file_path = "C:/Users/Natty PC/Documents/Party/Project II/Data/IoTScenarios/CTU-IoT-Malware-Capture-3-1/bro/conn.log.txt"
# model_path = "C:/Users/Natty PC/Documents/Party/Project II/Models/Model - DCT.joblib"
# process_log_in_intervals(log_file_path, model_path, interval_minutes=5)


In [ ]:
# import pandas as pd

# def check_log_file(file_path):
#     try:
#         # Load the log file into a DataFrame while skipping the first 8 rows and bad lines
#         df = pd.read_csv(file_path, delim_whitespace=True, header=None, na_filter=False, 
#                          on_bad_lines='skip', skiprows=8)

#         # Display the first few rows and the number of columns
#         print(f"Loaded DataFrame with {df.shape[0]} records and {df.shape[1]} columns.")
#         print(df.head())  # Show the first few rows
#     except Exception as e:
#         print(f"An error occurred: {e}")

# # Example usage
# log_file_path = "C:/Users/Natty PC/Documents/Party/Project II/Data/IoTScenarios/CTU-IoT-Malware-Capture-3-1/bro/conn.log.txt"
# check_log_file(log_file_path)


In [ ]:
# import pandas as pd
# import numpy as np
# import joblib

# def generate_signatures_from_log(df):
#     """
#     Generate signatures from the DataFrame of logs in the form [signature, frequency].
    
#     Parameters:
#         df (DataFrame): DataFrame containing the log data.
        
#     Returns:
#         list: A list of signatures and their frequencies.
#     """
#     # Initialize a dictionary to hold the frequency of each signature
#     signature_counts = {}

#     # Create signatures based on specified criteria
#     for _, row in df.iterrows():
#         # Extracting necessary fields
#         id_resp_p = row[5]  # id.resp_p
#         proto = row[6]  # proto
#         service = row[7]  # service
#         orig_bytes = row[9]  # orig_bytes
#         resp_bytes = row[10]  # resp_bytes
#         conn_state = row[11]  # conn_state
        
#         # Skip service if it's '0', '-', or empty
#         service = service if service not in ['0', '-', ''] else ''

#         # Replace '-' or '0' with '0' for orig_bytes and resp_bytes
#         orig_bytes = orig_bytes if orig_bytes not in ['-', '0'] else '0'
#         resp_bytes = resp_bytes if resp_bytes not in ['-', '0'] else '0'

#         # Create the signature string
#         signature = f"{id_resp_p}{proto}{service}{orig_bytes}{resp_bytes}{conn_state}"
        
#         # Count frequencies
#         if signature in signature_counts:
#             signature_counts[signature] += 1
#         else:
#             signature_counts[signature] = 1

#     # Prepare the output list in the desired format: [[signature, frequency], ...]
#     output_list = [[signature, frequency] for signature, frequency in signature_counts.items()]

#     return output_list  # Return a list instead of a DataFrame for the output

# def process_log_in_intervals(file_path, model_path, interval_minutes=5):
#     try:
#         # Load the log file into a DataFrame while skipping the first 8 rows
#         df = pd.read_csv(file_path, delim_whitespace=True, header=None, na_filter=False,
#                          on_bad_lines='skip', skiprows=8, low_memory=False)

#         # Replace non-numeric values in the timestamp column with NaN and convert
#         df[0] = pd.to_numeric(df[0], errors='coerce')  # Convert timestamps to numeric
        
#         # Drop rows with NaN values in critical columns
#         df.dropna(subset=[0, 3, 5, 6, 14, 16, 17], inplace=True)  # Ensure key columns are not NaN

#         # Convert timestamp to datetime
#         df[0] = pd.to_datetime(df[0], errors='coerce')  # Convert to datetime

#         # Check for missing values after conversion
#         df.dropna(subset=[0], inplace=True)

#         # Load the trained model
#         model_data = joblib.load(model_path)
#         clf = model_data['model']
#         signature_list = model_data['signature_list']

#         # Define time intervals
#         start_time = df[0].min()
#         end_time = df[0].max()
#         current_time = start_time

#         while current_time <= end_time:
#             # Create time range for the current interval
#             next_time = current_time + pd.Timedelta(minutes=interval_minutes)

#             # Filter the DataFrame for the current time range
#             filtered_df = df[(df[0] >= current_time) & (df[0] < next_time)]

#             if not filtered_df.empty:
#                 # Generate signatures from the filtered DataFrame
#                 signatures = generate_signatures_from_log(filtered_df)

#                 # Prepare the signature matrix for prediction
#                 X_matrix = np.zeros((1, len(signature_list)), dtype=int)

#                 # Populate the matrix with frequencies
#                 for sig, frequency in signatures:
#                     if sig in signature_list:
#                         sig_idx = signature_list.index(sig)  # Get the index for the signature
#                         X_matrix[0, sig_idx] += frequency  # Update the corresponding cell with the frequency

#                 # Make predictions
#                 prediction = clf.predict(X_matrix)

#                 # Print results
#                 print(f"Time Interval: {current_time} - {next_time}")
#                 print(f"Signatures: {signatures}")
#                 print(f"Predicted Label: {prediction[0]}")

#             # Move to the next time interval
#             current_time = next_time

#     except FileNotFoundError:
#         print(f"File not found: {file_path} or {model_path}")
#     except Exception as e:
#         print(f"An error occurred: {e}")

# # Example usage
# log_file_path = "C:/Users/Natty PC/Documents/Party/Project II/Data/IoTScenarios/CTU-IoT-Malware-Capture-3-1/bro/conn.log.txt"
# model_path = "C:/Users/Natty PC/Documents/Party/Project II/Models/Model - DCT.joblib"
# process_log_in_intervals(log_file_path, model_path, interval_minutes=5)


In [92]:
# import pandas as pd
# import numpy as np
# import joblib

# def generate_signatures_from_log(df):
#     """
#     Generate signatures from the DataFrame of logs in the form [signature, frequency].
    
#     Parameters:
#         df (DataFrame): DataFrame containing the log data.
        
#     Returns:
#         list: A list of signatures and their frequencies.
#     """
#     # Initialize a dictionary to hold the frequency of each signature
#     signature_counts = {}

#     # Create signatures based on specified criteria
#     for _, row in df.iterrows():
#         # Extracting necessary fields
#         id_resp_p = row[5]  # id.resp_p
#         proto = row[6]  # proto
#         service = row[7]  # service
#         orig_bytes = row[9]  # orig_bytes
#         resp_bytes = row[10]  # resp_bytes
#         conn_state = row[11]  # conn_state
        
#         # Skip service if it's '0', '-', or empty
#         service = service if service not in ['0', '-', ''] else ''

#         # Replace '-' or '0' with '0' for orig_bytes and resp_bytes
#         orig_bytes = orig_bytes if orig_bytes not in ['-', '0'] else '0'
#         resp_bytes = resp_bytes if resp_bytes not in ['-', '0'] else '0'

#         # Create the signature string
#         signature = f"{id_resp_p}{proto}{service}{orig_bytes}{resp_bytes}{conn_state}"
        
#         # Count frequencies
#         if signature in signature_counts:
#             signature_counts[signature] += 1
#         else:
#             signature_counts[signature] = 1

#     # Prepare the output list in the desired format: [[signature, frequency], ...]
#     output_list = [[signature, frequency] for signature, frequency in signature_counts.items()]

#     return output_list  # Return a list instead of a DataFrame for the output

# def process_log_in_intervals(file_path, model_path, interval_minutes=5):
#     try:
#         # Load the log file into a DataFrame while skipping the first 8 rows
#         df = pd.read_csv(file_path, delim_whitespace=True, header=None, na_filter=False,
#                          on_bad_lines='skip', skiprows=8, low_memory=False)

#         # Replace non-numeric values in the timestamp column with NaN and convert
#         df[0] = pd.to_numeric(df[0], errors='coerce')  # Convert timestamps to numeric
        
#         # Drop rows with NaN values in critical columns
#         df.dropna(subset=[0, 3, 5, 6, 14, 16, 17], inplace=True)  # Ensure key columns are not NaN

#         # Convert timestamp to datetime
#         df[0] = pd.to_datetime(df[0], errors='coerce')  # Convert to datetime

#         # Check for missing values after conversion
#         df.dropna(subset=[0], inplace=True)

#         # Load the trained model
#         model_data = joblib.load(model_path)
#         clf = model_data['model']
#         signature_list = model_data['signature_list']

#         # Define time intervals
#         start_time = df[0].min()
#         end_time = df[0].max()
#         current_time = start_time

#         all_predictions = {}  # Dictionary to store predictions for all intervals

#         print(f"Processing intervals from {start_time} to {end_time}")  # Debugging line

#         while current_time < end_time:
#             # Create time range for the current interval
#             next_time = current_time + pd.Timedelta(minutes=interval_minutes)

#             # Filter the DataFrame for the current time range
#             filtered_df = df[(df[0] >= current_time) & (df[0] < next_time)]

#             # Debugging output to see the current time range and filtered results
#             print(f"Current Time: {current_time}, Next Time: {next_time}, Filtered Count: {len(filtered_df)}")

#             if not filtered_df.empty:
#                 # Generate signatures from the filtered DataFrame
#                 signatures = generate_signatures_from_log(filtered_df)

#                 # Debugging: Print generated signatures
#                 print(f"Generated Signatures: {signatures}")

#                 # Prepare the signature matrix for prediction
#                 X_matrix = np.zeros((1, len(signature_list)), dtype=int)

#                 # Populate the matrix with frequencies
#                 for sig, frequency in signatures:
#                     if sig in signature_list:
#                         sig_idx = signature_list.index(sig)  # Get the index for the signature
#                         X_matrix[0, sig_idx] += frequency  # Update the corresponding cell with the frequency

#                 # Make predictions
#                 predictions = clf.predict(X_matrix)

#                 # Check if predictions are an array (e.g., multiple classes)
#                 if isinstance(predictions, np.ndarray):
#                     predictions = predictions.tolist()

#                 # Debugging: Print predictions
#                 print(f"Predictions: {predictions}")

#                 # Store the prediction in the dictionary with the current time interval as the key
#                 time_interval_key = f"{current_time} - {next_time}"
#                 if time_interval_key in all_predictions:
#                     all_predictions[time_interval_key].extend(predictions)  # Append new predictions
#                 else:
#                     all_predictions[time_interval_key] = predictions  # Store new prediction

#             # Move to the next time interval
#             current_time = next_time  # Update to the next interval

#         # Print all predictions after processing all intervals
#         print("All Predictions:")
#         for time_interval, predicted_labels in all_predictions.items():
#             # Convert to a set to remove duplicates, then back to a list and sort
#             unique_labels = sorted(set(predicted_labels))  
#             print(f"Time Interval: {time_interval} : {unique_labels}")

#     except FileNotFoundError:
#         print(f"File not found: {file_path} or {model_path}")
#     except Exception as e:
#         print(f"An error occurred: {e}")

In [147]:
# import pandas as pd
# import numpy as np
# import joblib

# def print_log_duration(file_path, nrows=None):
#     """
#     Print the start time, end time, and duration of the dataset.
#     """
#     try:
#         # Read the log file into a DataFrame
#         df = pd.read_table(file_path, delim_whitespace=True, header=None, usecols=[0], skiprows=8, nrows=nrows)
        
#         # Convert the first column (timestamp) to datetime
#         df.columns = ['Timestamp']
#         df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce', unit='s')
        
#         # Calculate the duration by subtracting the minimum timestamp from the maximum timestamp
#         min_timestamp = df['Timestamp'].min()
#         max_timestamp = df['Timestamp'].max()
#         duration = max_timestamp - min_timestamp
        
#         # Print the start time, end time, and duration
#         print(f"Start Time: {min_timestamp}")
#         print(f"End Time:   {max_timestamp}")
#         print(f"Duration: {duration}")
        
#     except FileNotFoundError:
#         print(f"File not found: {file_path}")
#     except Exception as e:
#         print(f"An error occurred: {e}")

# def generate_signatures_from_log(df):
#     """
#     Generate signatures from the DataFrame of logs in the form [signature, frequency].
#     """
#     signature_counts = {}
    
#     for _, row in df.iterrows():
#         id_resp_p = row[5]
#         proto = row[6]
#         service = row[7]
#         orig_bytes = row[9]
#         resp_bytes = row[10]
#         conn_state = row[11]
        
#         service = service if service not in ['0', '-', ''] else ''
#         orig_bytes = orig_bytes if orig_bytes not in ['-', '0'] else '0'
#         resp_bytes = resp_bytes if resp_bytes not in ['-', '0'] else '0'

#         signature = f"{id_resp_p}{proto}{service}{orig_bytes}{resp_bytes}{conn_state}"

#         if signature in signature_counts:
#             signature_counts[signature] += 1
#         else:
#             signature_counts[signature] = 1

#     output_list = [[signature, frequency] for signature, frequency in signature_counts.items()]
#     return output_list

# def process_log_in_intervals(file_path, model_path, interval_minutes=5):
#     try:
#         # Print the duration of the log file
#         print("Scanning dataset duration:")
#         print_log_duration(file_path)  # Using the provided function to print duration

#         # Load the log data into a DataFrame
#         df = pd.read_csv(file_path, delim_whitespace=True, header=None, na_filter=False,
#                          on_bad_lines='skip', skiprows=8, low_memory=False)

#         # Convert the first column to datetime (timestamps)
#         df[0] = pd.to_numeric(df[0], errors='coerce')
#         df.dropna(subset=[0, 3, 5, 6, 14, 16, 17], inplace=True)
#         df[0] = pd.to_datetime(df[0], errors='coerce', unit='s')  # Ensure the timestamps are correct
#         df.dropna(subset=[0], inplace=True)

#         # Load the model and signature list
#         model_data = joblib.load(model_path)
#         clf = model_data['model']
#         signature_list = model_data['signature_list']

#         # Define time intervals
#         start_time = df[0].min()
#         end_time = df[0].max()
#         current_time = start_time

#         # Print the range of timestamps
#         print(f"Data Range: Min Timestamp = {start_time}, Max Timestamp = {end_time}")

#         # Process data in 5-minute intervals
#         interval_duration = pd.Timedelta(minutes=interval_minutes)
#         interval_count = 1  # Initialize interval counter

#         # Store row counts for each interval
#         interval_row_counts = []

#         while current_time < end_time:
#             next_time = current_time + interval_duration

#             # Filter data for the current 5-minute interval
#             filtered_df = df[(df[0] >= current_time) & (df[0] < next_time)]
            
#             # Count and print the number of rows in this interval
#             num_rows = len(filtered_df)
#             interval_row_counts.append((current_time, next_time, num_rows))  # Store the interval count

#             if num_rows > 0:  # Process only if there are rows in the interval
#                 # Generate signatures from the filtered logs
#                 signatures = generate_signatures_from_log(filtered_df)

#                 # Prepare the feature matrix (X_matrix) for prediction
#                 X_matrix = np.zeros((1, len(signature_list)), dtype=int)

#                 # Map signatures to the matrix
#                 for sig, frequency in signatures:
#                     if sig in signature_list:
#                         sig_idx = signature_list.index(sig)
#                         X_matrix[0, sig_idx] += frequency

#                 # Make predictions for the current interval
#                 predictions = clf.predict(X_matrix)

#                 # Ensure predictions are in list format
#                 if isinstance(predictions, np.ndarray):
#                     predictions = predictions.tolist()

#                 # Print the predictions for the current interval
#                 print(f"Time Interval {interval_count}: {current_time} - {next_time} : {predictions}")

#             # Move to the next 5-minute interval
#             current_time = next_time
#             interval_count += 1  # Increment interval counter

#         # After processing, print the row counts for each interval
#         print("\nRow counts for each interval:")
#         for start, end, count in interval_row_counts:
#             print(f"From {start} to {end}: {count} packets")

#         print("Processing complete.")

#     except FileNotFoundError:
#         print(f"File not found: {file_path} or {model_path}")
#     except Exception as e:
#         print(f"An error occurred: {e}")

# # Example usage
# # log_file_path = "path_to_your_log_file"
# # model_path = "path_to_your_model_file"
# # process_log_in_intervals(log_file_path, model_path, interval_minutes=5)


In [4]:
# import pandas as pd
# import numpy as np
# import joblib

# def print_log_duration(file_path, nrows=None):
#     """
#     Print the start time, end time, and duration of the dataset.
#     """
#     try:
#         # Read the log file into a DataFrame
#         df = pd.read_table(file_path, delim_whitespace=True, header=None, usecols=[0], skiprows=8, nrows=nrows)
        
#         # Convert the first column (timestamp) to datetime
#         df.columns = ['Timestamp']
#         df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce', unit='s')
        
#         # Calculate the duration by subtracting the minimum timestamp from the maximum timestamp
#         min_timestamp = df['Timestamp'].min()
#         max_timestamp = df['Timestamp'].max()
#         duration = max_timestamp - min_timestamp
        
#         # Print the start time, end time, and duration
#         print(f"Start Time: {min_timestamp}")
#         print(f"End Time:   {max_timestamp}")
#         print(f"Duration: {duration}")
        
#     except FileNotFoundError:
#         print(f"File not found: {file_path}")
#     except Exception as e:
#         print(f"An error occurred: {e}")

# def generate_signatures_from_log(df):
#     """
#     Generate signatures from the DataFrame of logs in the form [signature, frequency].
#     """
#     signature_counts = {}
    
#     for _, row in df.iterrows():
#         id_resp_p = row[5]
#         proto = row[6]
#         service = row[7]
#         orig_bytes = row[9]
#         resp_bytes = row[10]
#         conn_state = row[11]
        
#         service = service if service not in ['0', '-', ''] else ''
#         orig_bytes = orig_bytes if orig_bytes not in ['-', '0'] else '0'
#         resp_bytes = resp_bytes if resp_bytes not in ['-', '0'] else '0'

#         signature = f"{id_resp_p}{proto}{service}{orig_bytes}{resp_bytes}{conn_state}"

#         if signature in signature_counts:
#             signature_counts[signature] += 1
#         else:
#             signature_counts[signature] = 1

#     output_list = [[signature, frequency] for signature, frequency in signature_counts.items()]
#     return output_list

# def process_log_in_intervals(file_path, model_path, interval_minutes=1):
#     try:
#         # Print the duration of the log file
#         print("Scanning dataset duration:")
#         print_log_duration(file_path)  # Using the provided function to print duration

#         # Load the log data into a DataFrame
#         df = pd.read_csv(file_path, delim_whitespace=True, header=None, na_filter=False,
#                          on_bad_lines='skip', skiprows=8, low_memory=False)

#         # Convert the first column to datetime (timestamps)
#         df[0] = pd.to_numeric(df[0], errors='coerce')
#         df.dropna(subset=[0, 3, 5, 6, 14, 16, 17], inplace=True)
#         df[0] = pd.to_datetime(df[0], errors='coerce', unit='s')  # Ensure the timestamps are correct
#         df.dropna(subset=[0], inplace=True)

#         # Load the model and signature list
#         model_data = joblib.load(model_path)
#         clf = model_data['model']
#         signature_list = model_data['signature_list']

#         # Define time intervals
#         start_time = df[0].min()
#         end_time = df[0].max()
#         current_time = start_time

#         # Print the range of timestamps
#         print(f"Data Range: Min Timestamp = {start_time}, Max Timestamp = {end_time}")

#         # Process data in 5-minute intervals
#         interval_duration = pd.Timedelta(minutes=interval_minutes)
#         interval_count = 1  # Initialize interval counter

#         # Store row counts for each interval
#         interval_row_counts = []

#         # Label mapping
#         label_mapping = {
#             0: 'Attack',
#             1: 'Benign',
#             2: 'C&C',
#             3: 'C&C-HeartBeat',
#             4: 'DDoS',
#             5: 'Okiru',
#             6: 'PortScan',
#             7: 'PortScan-Attack'
#         }

#         while current_time < end_time:
#             next_time = current_time + interval_duration

#             # Filter data for the current 5-minute interval
#             filtered_df = df[(df[0] >= current_time) & (df[0] < next_time)]
            
#             # Count the number of rows in this interval
#             num_rows = len(filtered_df)
#             interval_row_counts.append((current_time, next_time, num_rows))  # Store the interval count

#             if num_rows > 0:  # Process only if there are rows in the interval
#                 # Generate signatures from the filtered logs
#                 signatures = generate_signatures_from_log(filtered_df)

#                 # Prepare the feature matrix (X_matrix) for prediction
#                 X_matrix = np.zeros((1, len(signature_list)), dtype=int)

#                 # Map signatures to the matrix
#                 for sig, frequency in signatures:
#                     if sig in signature_list:
#                         sig_idx = signature_list.index(sig)
#                         X_matrix[0, sig_idx] += frequency

#                 # Make predictions for the current interval
#                 predictions = clf.predict(X_matrix)

#                 # Convert numerical predictions to corresponding labels
#                 predicted_labels = [label_mapping[pred] for pred in predictions]

#                 # Print the output in the desired format
#                 signatures_output = ', '.join([f'[{sig}, {freq}]' for sig, freq in signatures])
#                 print(f"Interval {interval_count}: {signatures_output}")
#                 print(f">> prediction: {predicted_labels}")

#             # Move to the next 5-minute interval
#             current_time = next_time
#             interval_count += 1  # Increment interval counter

#         print("Processing complete.")

#     except FileNotFoundError:
#         print(f"File not found: {file_path} or {model_path}")
#     except Exception as e:
#         print(f"An error occurred: {e}")


Work with DCT model

In [25]:
# # Version 1
# import pandas as pd
# import numpy as np
# import joblib

# # Function to generate signatures from a DataFrame
# def generate_signatures_from_log(df):
#     signature_counts = {}
    
#     for _, row in df.iterrows():
#         id_resp_p = row[5]
#         proto = row[6]
#         service = row[7]
#         orig_bytes = row[9]
#         resp_bytes = row[10]
#         conn_state = row[11]
        
#         service = service if service not in ['0', '-', ''] else ''
#         orig_bytes = orig_bytes if orig_bytes not in ['-', '0'] else '0'
#         resp_bytes = resp_bytes if resp_bytes not in ['-', '0'] else '0'

#         signature = f"{id_resp_p}{proto}{service}{orig_bytes}{resp_bytes}{conn_state}"

#         if signature in signature_counts:
#             signature_counts[signature] += 1
#         else:
#             signature_counts[signature] = 1

#     output_list = [[signature, frequency] for signature, frequency in signature_counts.items()]
#     return output_list

# def process_log_in_intervals(file_path, model_path, interval_minutes=5):
#     try:
#         # Print the duration of the log file
#         print("Scanning dataset duration:")
#         # print_log_duration(file_path)  # Using the provided function to print duration

#         # Load the log data into a DataFrame
#         df = pd.read_csv(file_path, delim_whitespace=True, header=None, na_filter=False,
#                          on_bad_lines='skip', skiprows=8, low_memory=False)

#         # Convert the first column to datetime (timestamps)
#         df[0] = pd.to_numeric(df[0], errors='coerce')
#         df.dropna(subset=[0, 3, 5, 6, 14, 16, 17], inplace=True)
#         df[0] = pd.to_datetime(df[0], errors='coerce', unit='s')  # Ensure the timestamps are correct
#         df.dropna(subset=[0], inplace=True)

#         # Load the model and signature list
#         model_data = joblib.load(model_path)
#         clf = model_data['model']
#         signature_list = model_data['signature_list']

#         # Define time intervals
#         start_time = df[0].min()
#         end_time = df[0].max()
#         current_time = start_time

#         # Print the range of timestamps
#         print(f"Data Range: Min Timestamp = {start_time}, Max Timestamp = {end_time}")

#         # Process data in 5-minute intervals
#         interval_duration = pd.Timedelta(minutes=interval_minutes)
#         interval_count = 1  # Initialize interval counter

#         # Label mapping
#         label_mapping = {
#             0: 'Attack',
#             1: 'C&C',
#             2: 'C&C-HeartBeat',
#             3: 'C&C-Torii',
#             4: 'DDoS',
#             5: 'Okiru',
#             6: 'PortScan',
#             7: 'PortScan-Attack'
#         }

#         while current_time < end_time:
#             next_time = current_time + interval_duration

#             # Filter data for the current 5-minute interval
#             filtered_df = df[(df[0] >= current_time) & (df[0] < next_time)]

#             if not filtered_df.empty:  # Process only if there are rows in the interval
#                 # Generate signatures from the filtered logs
#                 signatures = generate_signatures_from_log(filtered_df)

#                 # Prepare the feature matrix (X_matrix) for prediction
#                 X_matrix = np.zeros((len(signatures), len(signature_list)), dtype=int)

#                 # Map signatures to the matrix
#                 for sig, frequency in signatures:
#                     if sig in signature_list:
#                         sig_idx = signature_list.index(sig)
#                         # Update the matrix for this signature
#                         X_matrix[signatures.index([sig, frequency]), sig_idx] += frequency

#                 # Make predictions for all signatures in the current interval
#                 predictions = clf.predict(X_matrix)

#                 # Convert numerical predictions to corresponding labels
#                 predicted_labels = [label_mapping[pred] for pred in predictions]

#                 # Remove duplicates from predicted_labels
#                 unique_predicted_labels = list(set(predicted_labels))

#                 # Print the output in the desired format
#                 signatures_output = ', '.join([f'[{sig}, {freq}]' for sig, freq in signatures])
#                 # print(f"Interval {interval_count}: {signatures_output}")
#                 print(f"Interval {interval_count}: {unique_predicted_labels}")
#                 # print(f">> Unique predictions: {unique_predicted_labels}")

#             # Move to the next 5-minute interval
#             current_time = next_time
#             interval_count += 1  # Increment interval counter

#         print("Processing complete.")

#     except FileNotFoundError:
#         print(f"File not found: {file_path} or {model_path}")
#     except Exception as e:
#         print(f"An error occurred: {e}")


Work with all model 7 datasets

In [28]:
# # Version 2  Can predict all dataset & all models
# import pandas as pd
# import numpy as np
# import joblib

# # Function to generate signatures from a DataFrame
# def generate_signatures_from_log(df):
#     signature_counts = {}
    
#     for _, row in df.iterrows():
#         id_resp_p = row[5]
#         proto = row[6]
#         service = row[7]
#         orig_bytes = row[9]
#         resp_bytes = row[10]
#         conn_state = row[11]
        
#         # Clean up service and byte values
#         service = service if service not in ['0', '-', ''] else ''
#         orig_bytes = orig_bytes if orig_bytes not in ['-', '0'] else '0'
#         resp_bytes = resp_bytes if resp_bytes not in ['-', '0'] else '0'

#         # Create the signature
#         signature = f"{id_resp_p}{proto}{service}{orig_bytes}{resp_bytes}{conn_state}"

#         # Count occurrences of each signature
#         signature_counts[signature] = signature_counts.get(signature, 0) + 1

#     output_list = [[signature, frequency] for signature, frequency in signature_counts.items()]
#     return output_list

# def process_log_in_intervals(file_path, model_path, interval_minutes):  # Changed to 1 minute
#     try:
#         # Load the log data into a DataFrame
#         df = pd.read_csv(file_path, delim_whitespace=True, header=None, na_filter=False,
#                          on_bad_lines='skip', skiprows=8, low_memory=False)

#         # Convert the first column to datetime (timestamps)
#         df[0] = pd.to_numeric(df[0], errors='coerce')
#         df.dropna(subset=[0, 3, 5, 6, 14, 16, 17], inplace=True)
#         df[0] = pd.to_datetime(df[0], errors='coerce', unit='s')  # Ensure the timestamps are correct
#         df.dropna(subset=[0], inplace=True)

#         # Load the model and signature list
#         model_data = joblib.load(model_path)
#         clf = model_data['model']
#         signature_list = model_data['signature_list']

#         # Define time intervals
#         start_time = df[0].min()
#         end_time = df[0].max()
#         current_time = start_time

#         # Print the range of timestamps
#         print(f"Data Range: Min Timestamp = {start_time}, Max Timestamp = {end_time}")

#         # Process data in defined intervals
#         interval_duration = pd.Timedelta(minutes=interval_minutes)
#         interval_count = 1  # Initialize interval counter

#         # Label mapping
#         label_mapping = {
#             0: 'Attack',
#             1: 'C&C',
#             2: 'C&C-HeartBeat',
#             3: 'C&C-Torii',
#             4: 'DDoS',
#             5: 'Okiru',
#             6: 'PortScan',
#             7: 'PortScan-Attack'
#         }

#         # List to capture predictions over time
#         predictions_over_time = []

#         while current_time < end_time:
#             next_time = current_time + interval_duration

#             # Filter data for the current interval
#             filtered_df = df[(df[0] >= current_time) & (df[0] < next_time)]

#             if not filtered_df.empty:  # Process only if there are rows in the interval
#                 # Generate signatures from the filtered logs
#                 signatures = generate_signatures_from_log(filtered_df)

#                 # Prepare the feature matrix (X_matrix) for prediction
#                 X_matrix = np.zeros((len(signatures), len(signature_list)), dtype=int)

#                 # Map signatures to the matrix
#                 for i, (sig, frequency) in enumerate(signatures):
#                     if sig in signature_list:
#                         sig_idx = signature_list.index(sig)
#                         # Update the matrix for this signature
#                         X_matrix[i, sig_idx] = frequency  # Use i to index X_matrix

#                 # Make predictions for all signatures in the current interval
#                 predictions = clf.predict(X_matrix)

#                 # Convert numerical predictions to corresponding labels
#                 predicted_labels = [label_mapping[pred] for pred in predictions]

#                 # Store predictions for the current time interval
#                 predictions_over_time.append((current_time, predicted_labels))

#                 # Print the unique predicted labels for the interval
#                 unique_predicted_labels = list(set(predicted_labels))
#                 # print(f"Interval {interval_count}: {unique_predicted_labels}")

#                 # Count frequency of each prediction in this interval
#                 prediction_counts = {label: predicted_labels.count(label) for label in unique_predicted_labels}
#                 print(f"Interval {interval_count}: \t{current_time}")
#                 print(f"\t\t{prediction_counts}")

#             # Move to the next interval
#             current_time = next_time
#             interval_count += 1  # Increment interval counter

#         print("Processing complete.")

#         # After processing, print predictions for each time interval
#         # for time, preds in predictions_over_time:
#         #     print(f"Time: {time}, Predictions: {preds}")

#     except FileNotFoundError:
#         print(f"File not found: {file_path} or {model_path}")
#     except Exception as e:
#         print(f"An error occurred: {e}")


Optimize code

In [64]:
import pandas as pd
import numpy as np
import joblib
import psutil  # Library for monitoring system resources
import time

# Function to generate signatures from a DataFrame
def generate_signatures_from_log(df):
    signature_counts = {}

    for _, row in df.iterrows():
        id_resp_p = row[5]
        proto = row[6]
        service = row[7]
        orig_bytes = row[9]
        resp_bytes = row[10]
        conn_state = row[11]

        # Clean up service and byte values
        service = service if service not in ['0', '-', ''] else ''
        orig_bytes = orig_bytes if orig_bytes not in ['-', '0'] else '0'
        resp_bytes = resp_bytes if resp_bytes not in ['-', '0'] else '0'

        # Create the signature
        signature = f"{id_resp_p}{proto}{service}{orig_bytes}{resp_bytes}{conn_state}"

        # Count occurrences of each signature
        signature_counts[signature] = signature_counts.get(signature, 0) + 1

    output_list = [[signature, frequency] for signature, frequency in signature_counts.items()]
    return output_list

# Function to monitor and print resource usage for each batch
def measure_resource_usage(initial_memory, initial_cpu):
    process = psutil.Process()
    current_memory = process.memory_info().rss / (1024 * 1024)  # Memory usage in MB
    current_cpu = psutil.cpu_percent(interval=1)  # CPU usage in percentage

    # Calculate the difference (resource used by the current batch)
    memory_used = current_memory - initial_memory
    cpu_used = current_cpu - initial_cpu

    return memory_used, cpu_used

def process_log_in_intervals(file_path, model_path, interval_minutes=5, batch_size=5):
    try:
        # Load the model and signature list
        model_data = joblib.load(model_path)
        clf = model_data['model']
        signature_list = model_data['signature_list']

        # Define time intervals in seconds
        interval_duration = pd.Timedelta(minutes=interval_minutes)

        # Chunk size to read the file in batches
        chunk_size = 5000  # Adjust based on your system's RAM capacity

        # Read the log data in chunks
        df_iterator = pd.read_csv(file_path, delim_whitespace=True, header=None, na_filter=False,
                                  on_bad_lines='skip', skiprows=8, low_memory=False, chunksize=chunk_size)

        # Initialize variables to track current interval
        current_time = None
        next_time = None
        interval_count = 1

        # Initialize variables for batch processing
        interval_buffer = []
        time_buffer = []
        batch_counter = 0

        # Label mapping
        label_mapping = {
            0: 'Attack',
            1: 'C&C',
            2: 'C&C-HeartBeat',
            3: 'C&C-Torii',
            4: 'DDoS',
            5: 'Okiru',
            6: 'PortScan',
            7: 'PortScan-Attack'
        }

        # Measure the initial resource usage before starting processing
        print("Measuring Initial Resource Usage...")
        process = psutil.Process()                                                                                         # Get the current process
        initial_memory_usage = process.memory_info().rss / (1024 * 1024)                                                   # Initial memory usage in MB
        # initial_memory_usage = process.memory_info().rss / 1024                                                          # Initial memory usage in KB
        initial_cpu_usage = psutil.cpu_percent(interval=1)                                                                 # Initial CPU usage over 1 second
        print(f"Initial Memory Usage: {initial_memory_usage:.2f} MB, Initial CPU Usage: {initial_cpu_usage:.2f}%\n")

        start_time = time.time()  # Start time to measure execution duration

        for chunk in df_iterator:
            # Preprocess the chunk
            chunk[0] = pd.to_numeric(chunk[0], errors='coerce')
            chunk.dropna(subset=[0, 3, 5, 6, 14, 16, 17], inplace=True)
            chunk[0] = pd.to_datetime(chunk[0], errors='coerce', unit='s')  # Convert timestamp to datetime
            chunk.dropna(subset=[0], inplace=True)

            if current_time is None:
                current_time = chunk[0].min()
                next_time = current_time + interval_duration

            # Process the chunk in intervals
            while current_time < chunk[0].max():
                # Filter data for the current interval
                interval_df = chunk[(chunk[0] >= current_time) & (chunk[0] < next_time)]

                if not interval_df.empty:
                    # Generate signatures from the filtered logs
                    signatures = generate_signatures_from_log(interval_df)

                    # Prepare the feature matrix (X_matrix) for prediction
                    X_matrix = np.zeros((len(signatures), len(signature_list)), dtype=int)

                    # Map signatures to the matrix
                    for i, (sig, frequency) in enumerate(signatures):
                        if sig in signature_list:
                            sig_idx = signature_list.index(sig)
                            # Update the matrix for this signature
                            X_matrix[i, sig_idx] = frequency  # Use i to index X_matrix

                    # Add to interval buffer
                    interval_buffer.append(X_matrix)
                    time_buffer.append(current_time)
                    batch_counter += 1

                # If batch size is reached, make predictions for all intervals in the batch
                if batch_counter >= batch_size:
                    for X_mat, interval_time in zip(interval_buffer, time_buffer):
                        # Make predictions for all signatures in the current batch of intervals
                        predictions = clf.predict(X_mat)

                        # Convert numerical predictions to corresponding labels
                        predicted_labels = [label_mapping[pred] for pred in predictions]

                        # Print the unique predicted labels for the interval
                        unique_predicted_labels = list(set(predicted_labels))
                        prediction_counts = {label: predicted_labels.count(label) for label in unique_predicted_labels}
                        print(f"Interval {interval_count}: {interval_time}")
                        print(f"\tPredictions: {prediction_counts}")

                        interval_count += 1

                    # Clear buffers after processing batch
                    interval_buffer = []
                    time_buffer = []
                    batch_counter = 0

                    # Measure resource usage for the current batch and calculate the difference
                    memory_used, cpu_used = measure_resource_usage(initial_memory_usage, initial_cpu_usage)
                    # print(f"\nBatch {interval_count // batch_size} Resource Usage:")
                    print(f"------- Memory Used: {memory_used:.2f} MB\n------- CPU Used: {cpu_used:.2f}%\n")

                # Move to the next interval
                current_time = next_time
                next_time = current_time + interval_duration

        # Process any remaining intervals in the buffer
        if interval_buffer:
            for X_mat, interval_time in zip(interval_buffer, time_buffer):
                predictions = clf.predict(X_mat)
                predicted_labels = [label_mapping[pred] for pred in predictions]
                unique_predicted_labels = list(set(predicted_labels))
                prediction_counts = {label: predicted_labels.count(label) for label in unique_predicted_labels}
                print(f"Interval {interval_count}: {interval_time}")
                print(f"\tPredictions: {prediction_counts}")
                interval_count += 1

        # Measure final resource usage
        print("\nFinal Resource Usage:")
        memory_used, cpu_used = measure_resource_usage(initial_memory_usage, initial_cpu_usage)
        print(f"\tMemory Used: {memory_used:.2f} MB, CPU Used: {cpu_used:.2f}%\n")

        # Execution time
        end_time = time.time()
        print(f"\nTotal Execution Time: {end_time - start_time:.2f} seconds")

    except FileNotFoundError:
        print(f"File not found: {file_path} or {model_path}")
    except Exception as e:
        print(f"An error occurred: {e}")


# Log Data to choose
# log_file_path = "C:/Users/Natty PC/Documents/Party/Project II/Data/Capture-17-1.labeled"
# log_file_path = "C:/Users/Natty PC/Documents/Party/Project II/Data/Capture-3-1.labeled"
log_file_path = "C:/Users/Natty PC/Documents/Party/Project II/Data/Capture-34-1.labeled"
# log_file_path = "C:/Users/Natty PC/Documents/Party/Project II/Data/Capture-36-1.labeled"
# log_file_path = "C:/Users/Natty PC/Documents/Party/Project II/Data/Capture-60-1.labeled"
# log_file_path = "C:/Users/Natty PC/Documents/Party/Project II/Data/corrected_traffic_dataset.labeled"

# Trained Model to choose
model_path = "C:/Users/Natty PC/Documents/Party/Project II/Models/Model - DCT-mali.joblib"
# model_path = "C:/Users/Natty PC/Documents/Party/Project II/Models/Model - SVM-mali.joblib"
# model_path = "C:/Users/Natty PC/Documents/Party/Project II/Models/Model - RF-mali.joblib"
# model_path = "C:/Users/Natty PC/Documents/Party/Project II/Models/Model - AdaBoost-mali.joblib"

process_log_in_intervals(log_file_path, model_path, interval_minutes=5)


Optimize 2

In [ ]:
import pandas as pd
import numpy as np
import joblib
import psutil  # Library for monitoring system resources
import time
import logging
from concurrent.futures import ThreadPoolExecutor

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Function to generate signatures from a DataFrame
def generate_signatures_from_log(df):
    signature_counts = {}

    for _, row in df.iterrows():
        id_resp_p = row[5]
        proto = row[6]
        service = row[7]
        orig_bytes = row[9]
        resp_bytes = row[10]
        conn_state = row[11]

        # Clean up service and byte values
        service = service if service not in ['0', '-', ''] else ''
        orig_bytes = orig_bytes if orig_bytes not in ['-', '0'] else '0'
        resp_bytes = resp_bytes if resp_bytes not in ['-', '0'] else '0'

        # Create the signature
        signature = f"{id_resp_p}{proto}{service}{orig_bytes}{resp_bytes}{conn_state}"

        # Count occurrences of each signature
        signature_counts[signature] = signature_counts.get(signature, 0) + 1

    output_list = [[signature, frequency] for signature, frequency in signature_counts.items()]
    return output_list

# Function to monitor and print resource usage
def measure_resource_usage():
    process = psutil.Process()
    current_memory = process.memory_info().rss / (1024 * 1024)  # Memory usage in MB
    current_cpu = psutil.cpu_percent(interval=1)  # CPU usage in percentage
    return current_memory, current_cpu

# Function to dynamically allocate chunk size based on available memory
def get_dynamic_chunk_size():
    available_memory = psutil.virtual_memory().available / (1024 * 1024)  # in MB
    chunk_size = int(min(5000, max(1000, available_memory // 10)))  # scale with available memory
    logging.info(f"Dynamically set chunk size: {chunk_size}")
    return chunk_size

# Function to process a batch and log interval predictions
def process_batch(interval_buffer, time_buffer, clf, signature_list, label_mapping, batch_id):
    for idx, (X_mat, interval_time) in enumerate(zip(interval_buffer, time_buffer)):
        # Make predictions for all signatures in the current batch of intervals
        predictions = clf.predict(X_mat)

        # Convert numerical predictions to corresponding labels
        predicted_labels = [label_mapping[pred] for pred in predictions]

        # Print the unique predicted labels for the interval
        unique_predicted_labels = list(set(predicted_labels))
        prediction_counts = {label: predicted_labels.count(label) for label in unique_predicted_labels}
        logging.info(f"Batch {batch_id} Interval {interval_time}: {prediction_counts}")

# Main processing function
def process_log_in_intervals(file_path, model_path, interval_minutes=5, intervals_per_batch=5):
    try:
        # Load the model and signature list
        model_data = joblib.load(model_path)
        clf = model_data['model']
        signature_list = model_data['signature_list']

        # Label mapping
        label_mapping = {
            0: 'Attack',
            1: 'C&C',
            2: 'C&C-HeartBeat',
            3: 'C&C-Torii',
            4: 'DDoS',
            5: 'Okiru',
            6: 'PortScan',
            7: 'PortScan-Attack'
        }

        # Define time intervals in seconds
        interval_duration = pd.Timedelta(minutes=interval_minutes)

        # Dynamically set chunk size based on system resources
        chunk_size = get_dynamic_chunk_size()

        # Read the log data in chunks
        df_iterator = pd.read_csv(file_path, delim_whitespace=True, header=None, na_filter=False,
                                  on_bad_lines='skip', skiprows=8, low_memory=False, chunksize=chunk_size)

        # Initialize variables for batch processing
        current_time = None
        next_time = None
        interval_count = 1
        interval_buffer = []
        time_buffer = []
        interval_counter = 0  # Keep track of the number of intervals in a batch

        # Initialize resource usage variables
        initial_memory_usage, initial_cpu_usage = measure_resource_usage()
        max_memory_usage, max_cpu_usage = initial_memory_usage, initial_cpu_usage
        start_time = time.time()  # Start time for execution

        # Use ThreadPoolExecutor to process batches in parallel
        with ThreadPoolExecutor() as executor:
            for chunk in df_iterator:
                # Preprocess the chunk
                chunk[0] = pd.to_numeric(chunk[0], errors='coerce')
                chunk.dropna(subset=[0, 3, 5, 6, 14, 16, 17], inplace=True)
                chunk[0] = pd.to_datetime(chunk[0], errors='coerce', unit='s')  # Convert timestamp to datetime
                chunk.dropna(subset=[0], inplace=True)

                if current_time is None:
                    current_time = chunk[0].min()
                    next_time = current_time + interval_duration

                # Process the chunk in intervals
                while current_time < chunk[0].max():
                    interval_df = chunk[(chunk[0] >= current_time) & (chunk[0] < next_time)]

                    if not interval_df.empty:
                        # Generate signatures
                        signatures = generate_signatures_from_log(interval_df)

                        # Prepare feature matrix
                        X_matrix = np.zeros((len(signatures), len(signature_list)), dtype=int)
                        for i, (sig, frequency) in enumerate(signatures):
                            if sig in signature_list:
                                sig_idx = signature_list.index(sig)
                                X_matrix[i, sig_idx] = frequency

                        # Add to interval buffer
                        interval_buffer.append(X_matrix)
                        time_buffer.append(current_time)
                        interval_counter += 1

                    # Process a batch of intervals
                    if interval_counter >= intervals_per_batch:
                        batch_id = interval_count // intervals_per_batch

                        # Log predictions for each interval in the batch
                        executor.submit(process_batch, interval_buffer, time_buffer, clf, signature_list, label_mapping, batch_id)

                        # Measure resource usage after batch processing
                        current_memory_usage, current_cpu_usage = measure_resource_usage()
                        max_memory_usage = max(max_memory_usage, current_memory_usage)
                        max_cpu_usage = max(max_cpu_usage, current_cpu_usage)

                        logging.info(f"Batch {batch_id} Resource Usage: Memory Used: {current_memory_usage:.2f} MB, CPU Used: {current_cpu_usage:.2f}%")

                        # Clear buffers and reset interval counter after batch processing
                        interval_buffer, time_buffer = [], []
                        interval_counter = 0
                        interval_count += 1

                    current_time = next_time
                    next_time = current_time + interval_duration

            # Process any remaining intervals if they're less than the batch size
            if interval_buffer:
                batch_id = interval_count // intervals_per_batch
                process_batch(interval_buffer, time_buffer, clf, signature_list, label_mapping, batch_id)

        # Measure final resource usage
        current_memory_usage, current_cpu_usage = measure_resource_usage()
        max_memory_usage = max(max_memory_usage, current_memory_usage)
        max_cpu_usage = max(max_cpu_usage, current_cpu_usage)

        logging.info(f"Memory Used: {current_memory_usage:.2f} MB, CPU Used: {current_cpu_usage:.2f}%")

        # Execution time
        end_time = time.time()
        logging.info(f"Total Execution Time: {end_time - start_time:.2f} seconds")

        # Report maximum resource usage during the testing run
        max_memory_used = max_memory_usage - initial_memory_usage
        max_cpu_used = max_cpu_usage - initial_cpu_usage
        logging.info("\nFinal Resource Usage:")
        logging.info(f"Max Memory Used During Execution: {max_memory_usage:.2f} MB (Test Memory Usage: {max_memory_used:.2f} MB)")
        logging.info(f"Max CPU Used During Execution: {max_cpu_usage:.2f}% (Test CPU Usage: {max_cpu_used:.2f}%)")

    except FileNotFoundError:
        logging.error(f"File not found: {file_path} or {model_path}")
    except Exception as e:
        logging.error(f"An error occurred: {e}")


# Example Log Data to choose
# log_file_path = "C:/Users/Natty PC/Documents/Party/Project II/Data/Capture-17-1.labeled"
# log_file_path = "C:/Users/Natty PC/Documents/Party/Project II/Data/Capture-3-1.labeled"
log_file_path = "C:/Users/Natty PC/Documents/Party/Project II/Data/Capture-34-1.labeled"
# log_file_path = "C:/Users/Natty PC/Documents/Party/Project II/Data/Capture-36-1.labeled"
# log_file_path = "C:/Users/Natty PC/Documents/Party/Project II/Data/Capture-60-1.labeled"
# log_file_path = "C:/Users/Natty PC/Documents/Party/Project II/Data/corrected_traffic_dataset.labeled"

# Trained Model to choose
model_path = "C:/Users/Natty PC/Documents/Party/Project II/Models/Model - DCT-mali.joblib"
# model_path = "C:/Users/Natty PC/Documents/Party/Project II/Models/Model - SVM-mali.joblib"
# model_path = "C:/Users/Natty PC/Documents/Party/Project II/Models/Model - RF-mali.joblib"
# model_path = "C:/Users/Natty PC/Documents/Party/Project II/Models/Model - AdaBoost-mali.joblib"

# Measure initial resource usage
logging.info("Measuring Initial Resource Usage...")
initial_memory_usage, initial_cpu_usage = measure_resource_usage()
logging.info(f"Initial Memory Usage: {initial_memory_usage:.2f} MB, Initial CPU Usage: {initial_cpu_usage:.2f}%")

process_log_in_intervals(log_file_path, model_path, interval_minutes=15)


In [ ]:
# Log Data to choose
# log_file_path = "C:/Users/Natty PC/Documents/Party/Project II/Data/Capture-17-1.labeled"
# log_file_path = "C:/Users/Natty PC/Documents/Party/Project II/Data/Capture-3-1.labeled"
# log_file_path = "C:/Users/Natty PC/Documents/Party/Project II/Data/Capture-34-1.labeled"
# log_file_path = "C:/Users/Natty PC/Documents/Party/Project II/Data/Capture-36-1.labeled"
log_file_path = "C:/Users/Natty PC/Documents/Party/Project II/Data/Capture-60-1.labeled"
# log_file_path = "C:/Users/Natty PC/Documents/Party/Project II/Data/corrected_traffic_dataset.labeled"

# Trained Model to choose
model_path = "C:/Users/Natty PC/Documents/Party/Project II/Models/Model - DCT-mali.joblib"
# model_path = "C:/Users/Natty PC/Documents/Party/Project II/Models/Model - SVM-mali.joblib"
# model_path = "C:/Users/Natty PC/Documents/Party/Project II/Models/Model - RF-mali.joblib"
# model_path = "C:/Users/Natty PC/Documents/Party/Project II/Models/Model - AdaBoost-mali.joblib"

# Measure initial resource usage
logging.info("Measuring Initial Resource Usage...")
initial_memory_usage, initial_cpu_usage = measure_resource_usage()
logging.info(f"Initial Memory Usage: {initial_memory_usage:.2f} MB, Initial CPU Usage: {initial_cpu_usage:.2f}%")

process_log_in_intervals(log_file_path, model_path, interval_minutes=5)